In [1]:
import numpy as np   ## linear algebra
import pandas as pd  ## data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split  ## split train and test
from IPython.display import display ## Allows the use of display() for DataFrames

import matplotlib.pyplot as plt  ## data visualization

import gc   ## clean up 
import time ## time it

from subprocess import check_output
print(check_output(["ls", "./data"]).decode("utf8"))
# Any results you write to the current directory are saved as output.

aisles.csv
departments.csv
order_products__prior.csv
order_products__train.csv
orders.csv
products.csv
sample_submission.csv



In [3]:
def load_data(path_data):
    '''
    --------------------------------order_product--------------------------------
    * Unique in order_id + product_id
    '''
    priors = pd.read_csv(path_data + 'order_products__prior.csv', 
                     dtype={
                            'order_id': np.int32,
                            'product_id': np.uint16,
                            'add_to_cart_order': np.int16,
                            'reordered': np.int8})
    train = pd.read_csv(path_data + 'order_products__train.csv', 
                    dtype={
                            'order_id': np.int32,
                            'product_id': np.uint16,
                            'add_to_cart_order': np.int16,
                            'reordered': np.int8})
    '''
    --------------------------------order--------------------------------
    * This file tells us which set (prior, train, test) an order belongs
    * Unique in order_id
    * order_id in train, prior, test has no intersection
    * this is the #order_number order of this user
    '''
    orders = pd.read_csv(path_data + 'orders.csv', 
                         dtype={
                                'order_id': np.int32,
                                'user_id': np.int64,
                                'eval_set': 'category',
                                'order_number': np.int16,
                                'order_dow': np.int8,
                                'order_hour_of_day': np.int8,
                                'days_since_prior_order': np.float32})
    '''
    --------------------------------product--------------------------------
    * Unique in product_id
    '''
    products = pd.read_csv(path_data + 'products.csv')
    aisles = pd.read_csv(path_data + "aisles.csv")
    departments = pd.read_csv(path_data + "departments.csv")
    sample_submission = pd.read_csv(path_data + "sample_submission.csv")
    
    return priors, train, orders, products, aisles, departments, sample_submission


In [4]:
path_data = './data/'
df_priors, df_train, df_orders, df_products, df_aisles, df_departments, df_sample_submission = load_data(path_data)    

In [58]:
## Suggested by @Robert Goss https://www.kaggle.com/nickycan/lb-0-3805009-python-edition
import sys

def check_data(data):
    starting_size = sys.getsizeof(data)
    i = 0
    for c, dtype in zip(data.columns, data.dtypes):
        if 'int' in str(dtype):
            if min(data[c]) >=0:
                max_int =  max(data[c])
                if max_int <= 255:
                    data[c] = data[c].astype(np.uint8)
                elif max_int <= 65535:
                    data[c] = data[c].astype(np.uint16)
                elif max_int <= 4294967295:
                    data[c] = data[c].astype(np.uint32)
                i += 1
    print("Number of colums adjusted: {}\n".format(i))
    ## Changing known reorderd col to smaller int size
    data['reordered'] = np.nan_to_num(data['reordered']).astype(np.uint8)
    data.reordered.loc[data['reordered'] == 0] = np.nan  ## only replace such element in the column
    print("Reduced size {:.2%}".format(float(sys.getsizeof(data))/float(starting_size)))
    return data

In [57]:
my_check_list = [df_priors, df_train]
for this in my_check_list:
    check_data(this)
    

Number of colums adjusted: 3



/Users/udothemath1984/anaconda/lib/python3.6/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


Reduced size 100.00%
Number of colums adjusted: 3

Reduced size 100.00%


### The data is ready. ###